<a href="https://colab.research.google.com/github/omarhisham007/Deep-Learning/blob/main/gender_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.7.0.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/content/gender_input_output
! ls

/content/drive/MyDrive/content/gender_input_output
input  outputgender_model


In [4]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf

from google.colab.patches import cv2_imshow

from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPooling2D,AvgPool2D,GlobalAveragePooling2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split


from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


In [7]:
path = "/content/drive/MyDrive/content/gender_input_output/input/UTKFace"
pixels = []
age = []
gender = [] 

i=0
for img in os.listdir(path):
  i=i+1
  genders = img.split("_")[1]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img=cv2.resize(img,(100,100))
  pixels.append(np.array(img))
  gender.append(np.array(genders))

pixels = np.array(pixels)
gender = np.array(gender,np.uint64)

In [8]:
len(pixels)

23708

In [9]:
len(gender)

23708

In [10]:
x_train,x_test,y_train,y_test = train_test_split(pixels,gender,random_state=100)

In [11]:
len(x_train)

17781

In [12]:
len(x_test)

5927

In [13]:
input = Input(shape = (100,100,1))

conv1 = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(input)
conv1 = Dropout(0.1)(conv1)
conv1 = Activation('relu')(conv1)
pool1 = MaxPooling2D(pool_size = (2,2)) (conv1)

conv2 = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool1)
conv2 = Dropout(0.1)(conv2)
conv2 = Activation('relu')(conv2)
pool2 = MaxPooling2D(pool_size = (2,2)) (conv2)

conv3 = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool2)
conv3 = Dropout(0.1)(conv3)
conv3 = Activation('relu')(conv3)
pool3 = MaxPooling2D(pool_size = (2,2)) (conv3)

conv4 = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(pool3)
conv4 = Dropout(0.1)(conv4)
conv4 = Activation('relu')(conv4)
pool4 = MaxPooling2D(pool_size = (2,2)) (conv4)

flatten = Flatten()(pool4)

dense_1 = Dense(128,activation='relu')(flatten)

drop_1 = Dropout(0.2)(dense_1)

output = Dense(2,activation="sigmoid")(drop_1)

In [16]:
model = Model(inputs=input,outputs=output)
model.compile(optimizer="adam",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 dropout (Dropout)           (None, 100, 100, 32)      0         
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 64)        18496     
                                                           

In [19]:
model_path='./output/gender_model.h5'
checkpointer = ModelCheckpoint(model_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

In [20]:
save = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,callbacks=[callback_list])

Epoch 1/30
554/556 [============================>.] - ETA: 0s - loss: 0.3981 - accuracy: 0.8712
Epoch 00001: loss improved from inf to 0.39838, saving model to ./output/gender_model.h5
556/556 [==============================] - 13s 23ms/step - loss: 0.3984 - accuracy: 0.8711 - val_loss: 0.4013 - val_accuracy: 0.8643
Epoch 2/30
555/556 [============================>.] - ETA: 0s - loss: 0.3649 - accuracy: 0.8813
Epoch 00002: loss improved from 0.39838 to 0.36472, saving model to ./output/gender_model.h5
556/556 [==============================] - 13s 24ms/step - loss: 0.3647 - accuracy: 0.8813 - val_loss: 0.3583 - val_accuracy: 0.8805
Epoch 3/30
556/556 [==============================] - ETA: 0s - loss: 0.3422 - accuracy: 0.8893
Epoch 00003: loss improved from 0.36472 to 0.34217, saving model to ./output/gender_model.h5
556/556 [==============================] - 13s 23ms/step - loss: 0.3422 - accuracy: 0.8893 - val_loss: 0.3663 - val_accuracy: 0.8681
Epoch 4/30
556/556 [==================